# local_files

> Local files browser for importing external .db files

In [ ]:
#| default_exp components.local_files

In [ ]:
#| export
from typing import Any, List, Optional, Dict
from pathlib import Path
import json

from fasthtml.common import Div, Span, Button, Ul, Li, P
from cjm_fasthtml_interactions.core.context import InteractionContext

# DaisyUI components
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_sizes, btn_styles
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_sizes
from cjm_fasthtml_daisyui.components.feedback.alert import alert, alert_colors
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui, border_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius

# Tailwind utilities
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w, h, max_h, min_w, min_h
from cjm_fasthtml_tailwind.utilities.typography import (
    font_size, font_weight, font_family, text_align, truncate, list_style
)
from cjm_fasthtml_tailwind.utilities.layout import overflow
from cjm_fasthtml_tailwind.utilities.borders import border
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_direction, justify, items, gap, grow, shrink
)
from cjm_fasthtml_tailwind.core.base import combine_classes

# Lucide icons
from cjm_fasthtml_lucide_icons.factory import lucide_icon

# File browser library
from cjm_fasthtml_file_browser.core.config import (
    FileBrowserConfig, FilterConfig, ViewConfig, SelectionMode, ViewMode, FileColumn
)
from cjm_fasthtml_file_browser.core.models import BrowserState, BrowserSelection
from cjm_fasthtml_file_browser.providers.local import LocalFileSystemProvider
from cjm_fasthtml_file_browser.components.browser import render_file_browser

# Local imports
from cjm_transcript_source_select.html_ids import SelectionHtmlIds
from cjm_transcript_source_select.components.helpers import (
    _get_selection_state
)

## State Getters

In [ ]:
#| export
def _get_external_db_paths(
    ctx: InteractionContext  # Interaction context with state
) -> List[str]:  # List of external database paths
    """Get the list of external database paths from step state."""
    state = _get_selection_state(ctx)
    return state.get("external_db_paths", [])

In [ ]:
#| export
def _get_current_browse_path(
    ctx: InteractionContext  # Interaction context with state
) -> str:  # Current browse path
    """Get the current browse path from step state."""
    state = _get_selection_state(ctx)
    return state.get("current_browse_path", str(Path.home()))

In [ ]:
#| export
def _get_file_browser_state(
    step_state: Dict[str, Any],  # Selection step state dictionary
    default_path: Optional[str] = None  # Default path if no state exists
) -> BrowserState:  # BrowserState for file browser
    """Get or create BrowserState from step state."""
    state_dict = step_state.get("file_browser_state", {})
    if state_dict:
        return BrowserState.from_dict(state_dict)
    return BrowserState(current_path=default_path or str(Path.home()))

## File Browser Configuration

In [ ]:
#| export
def _create_db_browser_config() -> FileBrowserConfig:  # Configured FileBrowserConfig for .db file selection
    """Create file browser config for .db file selection."""
    return FileBrowserConfig(
        selection_mode=SelectionMode.SINGLE,
        selectable_types="files",
        view=ViewConfig(
            default_mode=ViewMode.LIST,
            allow_mode_toggle=False,
            columns=[FileColumn.NAME, FileColumn.SIZE, FileColumn.MODIFIED],
            sort_folders_first=True,
        ),
        filter=FilterConfig(
            allowed_extensions=[".db", ".sqlite", ".sqlite3"],
            show_directories=True,
            show_hidden=True,
        ),
        show_path_bar=True,
        show_breadcrumbs=True,
        show_toolbar=False,
        show_parent_navigation=True,
        # File browser's own container ID (separate from our wrapper)
        container_id="local-files-browser-inner",
        content_id="local-files-content",
    )

## External Sources List

In [ ]:
#| export
def _render_external_sources_list(
    external_paths: List[str],  # List of added external database paths
    remove_url: str,  # URL for removing external source
) -> Any:  # External sources list component
    """Render the list of added external database sources with scrollable paths."""
    if not external_paths:
        return None
    
    list_items = []
    for db_path in external_paths:
        path = Path(db_path)
        list_items.append(Li(
            Div(
                Span(path.name, cls=combine_classes(grow(), truncate, font_size.sm, font_family.mono)),
                Button(
                    lucide_icon("x", size=4, cls=str(text_dui.base_content.opacity(60))),
                    cls=combine_classes(btn, btn_styles.ghost, btn_sizes.xs, shrink._0),
                    hx_post=remove_url,
                    hx_vals=json.dumps({"db_path": db_path}),
                    # Use SOURCE_LIST ID for consistent zone focus ring
                    hx_target=SelectionHtmlIds.as_selector(SelectionHtmlIds.SOURCE_LIST),
                    hx_swap="outerHTML",
                    title=f"Remove {path.name}"
                ),
                cls=combine_classes(flex_display, items.center, gap(2), p.y(1), p.x(2))
            ),
            title=db_path,
            cls=combine_classes(border_dui.base_300, border.b(), bg_dui.primary.opacity(10))
        ))
    
    return Div(
        Div(
            Span("Added Sources", cls=str(font_weight.bold)),
            Span(
                str(len(external_paths)),
                cls=combine_classes(badge, badge_colors.primary, badge_sizes.sm, m.l(2))
            ),
            cls=combine_classes(
                flex_display, items.center,
                p(2), bg_dui.base_200.opacity(50), border_dui.base_300, border.b()
            )
        ),
        Ul(
            *list_items,
            id=SelectionHtmlIds.EXTERNAL_SOURCES_LIST,
            cls=combine_classes(list_style.none, m(0), p(0), overflow.y.auto, max_h(32))
        ),
        cls=combine_classes(m.t(2), shrink._0)
    )

## Local Files Browser

In [ ]:
#| export
def _render_local_files_browser(
    browser_state: Optional[BrowserState] = None,  # Current browser state
    external_paths: Optional[List[str]] = None,  # List of added external database paths
    provider: Optional[LocalFileSystemProvider] = None,  # File system provider
    config: Optional[FileBrowserConfig] = None,  # Browser configuration
    navigate_url: str = "",  # URL for browsing directories
    select_url: str = "",  # URL for adding external source (maps path to db_path)
    remove_url: str = "",  # URL for removing external source
    refresh_url: str = "",  # URL for refreshing browser
    path_input_url: str = "",  # URL for direct path input
    home_path: Optional[str] = None,  # Home directory path
    error_message: Optional[str] = None,  # Error message to display
) -> Any:  # Local files browser component
    """Render the local files browser for adding external .db files."""
    external_paths = external_paths or []
    
    # If no URLs provided, show placeholder
    if not navigate_url:
        return Div(
            P(
                "Local file browser not configured",
                cls=combine_classes(text_dui.base_content.opacity(50), text_align.center)
            ),
            P(
                "Missing navigation URL",
                cls=combine_classes(text_dui.base_content.opacity(30), text_align.center, font_size.sm)
            ),
            cls=combine_classes(
                p(8), flex_display, flex_direction.col, justify.center, items.center, h.full,
                border_radius.box
            ),
            # Use SOURCE_LIST ID for consistent zone focus ring with Plugin DB tab
            id=SelectionHtmlIds.SOURCE_LIST
        )
    
    # Create defaults if not provided
    if provider is None:
        provider = LocalFileSystemProvider()
    if config is None:
        config = _create_db_browser_config()
    if browser_state is None:
        browser_state = BrowserState(current_path=home_path or provider.get_home_path())
    if home_path is None:
        home_path = provider.get_home_path()
    
    # Get directory listing
    listing = provider.list_directory(browser_state.current_path)
    
    # Build error alert if present
    error_alert = None
    if error_message:
        error_alert = Div(
            Span(error_message, cls=str(font_size.sm)),
            role="alert",
            cls=combine_classes(alert, alert_colors.error, p(2), m.b(2))
        )
    
    # HTMX target for file browser - use SOURCE_LIST ID for consistent zone focus ring
    hx_target = SelectionHtmlIds.as_selector(SelectionHtmlIds.SOURCE_LIST)
    
    return Div(
        # Error message (if any)
        error_alert,
        # File browser from library (now first, takes available space)
        Div(
            render_file_browser(
                listing=listing,
                config=config,
                state=browser_state,
                navigate_url=navigate_url,
                select_url=select_url,
                toggle_view_url="",
                change_sort_url="",
                refresh_url=refresh_url,
                path_input_url=path_input_url,
                home_path=home_path,
                hx_target=hx_target,
            ),
            cls=combine_classes(grow(), overflow.y.auto, min_h._0)
        ),
        # Added external sources list (now below file browser)
        _render_external_sources_list(external_paths, remove_url),
        # Use SOURCE_LIST ID for consistent zone focus ring with Plugin DB tab
        id=SelectionHtmlIds.SOURCE_LIST,
        cls=combine_classes(
            w.full,
            h.full,
            min_w._0,
            flex_display,
            flex_direction.col,
            border_radius.box
        )
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()